In [1]:
import sys
sys.path.append('../')

from src.nn.linear import Linear
from src.nn.functional import ReLU, Softmax, MSELoss

import torch
from torch import nn
import torch.nn.functional as F

import numpy as np

import random
import matplotlib.pyplot as plt

%matplotlib inline

np.random.seed(54)

## Сравнение аутпутов Forward и Backward методов для всех реализованных слоев с аутпутам torch реализаций 

### Тесты линейного слоя

In [2]:
#init layers
torch_linear = torch.nn.Linear(3, 4, bias=True)

weigths = torch_linear.weight.data.numpy()
bias = torch_linear.bias.data.numpy()

my_linear = Linear(3, 4, bias=True)
my_linear.weights = weigths
my_linear.bias = bias

#test input
input_data = np.random.randn(3, 3).astype(np.float32)
loss_gradient = np.random.randn(3, 4).astype(np.float32)

#forward test
my_output = my_linear.forward(input_data)
torch_output = torch_linear(torch.tensor(input_data))

assert (my_output - torch_output.detach().numpy() < 1e-6).all() == True

#backward test
my_linear.backward(loss_gradient)
torch_output.backward(torch.tensor(loss_gradient))

assert (my_linear.weights_grad.T - torch_linear.weight.grad.detach().numpy() < 1e-6).all() == True
assert (my_linear.bias_grad.T - torch_linear.bias.grad.detach().numpy() < 1e-6).all() == True

In [3]:
print(my_linear.weights_grad.T, end='\n\n')
print(torch_linear.weight.grad.detach().numpy())

[[ 2.8958917   0.82432544 -0.86485815]
 [ 1.1170831   0.01014531 -1.0646278 ]
 [-3.3169525  -4.734642    1.0889834 ]
 [ 4.9491887   3.6359472  -1.2768121 ]]

[[ 2.8958917   0.82432544 -0.86485815]
 [ 1.1170831   0.01014531 -1.0646278 ]
 [-3.3169525  -4.734642    1.0889834 ]
 [ 4.9491887   3.6359472  -1.2768121 ]]


In [4]:
print(my_linear.bias_grad.T, end='\n\n')
print(torch_linear.bias.grad.detach().numpy())

[-1.8582057 -1.0067805  2.1648114 -3.0944514]

[-1.8582057 -1.0067805  2.1648114 -3.0944514]


### Тесты ReLU

In [5]:
#init layers
my_relu = ReLU()
torch_relu = torch.nn.ReLU()

#test input
input_data = np.random.randn(3, 4).astype(np.float32)
torch_input_data = torch.tensor(input_data, requires_grad=True)
loss_gradient = np.random.randn(3, 4).astype(np.float32)

#forward test
my_output = my_relu.forward(input_data)
torch_output = torch_relu(torch_input_data)

assert (my_output - torch_output.detach().numpy() < 1e-6).all() == True

#backward test
out_grad = my_relu.backward(loss_gradient)
torch_output.backward(torch.tensor(loss_gradient))

assert (out_grad - torch_input_data.grad.detach().numpy() < 1e-6).all() == True

In [6]:
print(out_grad, end='\n\n')
print(torch_input_data.grad.detach().numpy())

[[-0.         -0.         -0.45069039  0.14520353]
 [ 0.          0.          0.37093243  0.        ]
 [ 0.          0.          0.          0.        ]]

[[ 0.          0.         -0.4506904   0.14520353]
 [ 0.          0.          0.37093243  0.        ]
 [ 0.          0.          0.          0.        ]]


### Тесты Softmax

In [7]:
#test input
input_data = np.random.randn(3, 4).astype(np.float32)
torch_input_data = torch.tensor(input_data, requires_grad=True)
loss_gradient = np.random.randn(3, 4).astype(np.float32)

#init layers
my_softmax = Softmax()
torch_softmax = nn.Softmax(dim=1)

#forward test
my_output = my_softmax.forward(input_data)
torch_output = torch_softmax(torch_input_data)

assert (my_output - torch_output.detach().numpy() < 1e-6).all() == True

#backward test
out_grad = my_softmax.backward(loss_gradient)
torch_output.backward(torch.tensor(loss_gradient))

assert (out_grad - torch_input_data.grad.detach().numpy() < 1e-6).all() == True

In [8]:
print(my_output, end='\n\n')
print(torch_output.detach().numpy())

[[0.0641047  0.06651817 0.58650583 0.2828713 ]
 [0.35183212 0.11770014 0.12231466 0.4081531 ]
 [0.70372367 0.11998278 0.04972024 0.12657331]]

[[0.06410469 0.06651816 0.5865058  0.2828713 ]
 [0.3518321  0.11770012 0.12231466 0.40815306]
 [0.70372367 0.11998277 0.04972023 0.12657332]]


In [9]:
print(out_grad, end='\n\n')
print(torch_input_data.grad.detach().numpy())

[[-0.03177635 -0.17044323  0.33679397 -0.13457437]
 [ 0.47295687  0.04864965  0.00402987 -0.52563636]
 [-0.06725832 -0.02446857 -0.00678753  0.09851441]]

[[-0.03177634 -0.1704432   0.33679393 -0.13457437]
 [ 0.4729568   0.04864964  0.00402986 -0.5256364 ]
 [-0.06725833 -0.02446857 -0.00678753  0.09851442]]


### Тесты MSELoss

In [10]:
#init layers
my_mse = MSELoss()
torch_mse = torch.nn.MSELoss()

#test input
input_data = np.random.randn(3, 4).astype(np.float32) 
torch_input_data = torch.tensor(input_data, requires_grad=True)
labels = np.random.randn(3, 4).astype(np.float32) 

#forward test
my_output = my_mse(input_data, labels)
torch_output = torch_mse(torch_input_data, torch.tensor(labels))

assert (my_output - torch_output.item() < 1e-6).all() == True

#backward test
out_grad = my_mse.backward()
torch_output.backward()

assert (out_grad - torch_input_data.grad.numpy() < 1e-6).all() == True

In [11]:
print(my_output, end='\n\n')
print(torch_output.detach().numpy(), end='\n\n')

print(out_grad, end='\n\n')
print(torch_input_data.grad.numpy())

2.0901196

2.0901198

[[-0.17132546  0.16456425 -0.00071051 -0.12547956]
 [-0.60560256 -0.07275766  0.4123123  -0.1442021 ]
 [-0.17743908  0.00645827  0.05119051 -0.1659413 ]]

[[-0.17132547  0.16456425 -0.00071051 -0.12547956]
 [-0.6056026  -0.07275766  0.41231233 -0.14420211]
 [-0.17743908  0.00645827  0.05119052 -0.1659413 ]]
